In [1]:
from dask.distributed import Client, LocalCluster
from streamz import get_io_loop
import waterhealer as wh
import json

In [2]:
loop = get_io_loop() # it is better took loop for each program.

In [3]:
client = Client('tcp://127.0.0.1:61195', loop = loop)
client

Client Scheduler: tcp://127.0.0.1:61195 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [4]:
consumer_params = {
    'bootstrap.servers': 'localhost:9095, localhost:9096',
    'group.id': 'group-test-2',
    'auto.offset.reset': 'latest',
}

source = wh.from_kafka_batched(
    ['testing'],
    consumer_params, dask = True, loop = loop, maxlen = 10)

In [5]:
def json_loads(rows):
    results = []
    for row in rows:
        results.append((row[0], json.loads(row[1])))
    return results

def plus(rows):
    results = []
    for row in rows:
        no = row[1]['no']
        if no > 200:
            raise Exception('row is bigger than 50')
        results.append((row[0], no + 1))
    return results

In [6]:
outputs = []
source.map(json_loads).map(plus)\
.buffer(5).gather().partition(5)\
.foreach_async(wh.healing_batch, stream = source, asynchronous = True)\
.flatten().flatten().sink(outputs.append)

Output()

In [7]:
source.start()

TopicPartition{topic=testing,partition=0,offset=0,error=None}
TopicPartition{topic=testing,partition=0,offset=0,error=None} 349 339
TopicPartition{topic=testing,partition=1,offset=0,error=None}
TopicPartition{topic=testing,partition=1,offset=0,error=None} 315 305
TopicPartition{topic=testing,partition=2,offset=0,error=None}
TopicPartition{topic=testing,partition=2,offset=0,error=None} 335 325
TopicPartition{topic=testing,partition=3,offset=0,error=None}
TopicPartition{topic=testing,partition=3,offset=0,error=None} 303 293
TopicPartition{topic=testing,partition=4,offset=0,error=None}
TopicPartition{topic=testing,partition=4,offset=0,error=None} 306 297
TopicPartition{topic=testing,partition=5,offset=0,error=None}
TopicPartition{topic=testing,partition=5,offset=0,error=None} 281 272
TopicPartition{topic=testing,partition=6,offset=0,error=None}
TopicPartition{topic=testing,partition=6,offset=0,error=None} 295 286
TopicPartition{topic=testing,partition=7,offset=0,error=None}
TopicPartition

In [ ]:
len(outputs)

In [ ]:
outputs

In [ ]:
import streamz

In [ ]:
type(source) == streamz.dask.starmap

In [ ]:
from kafka import KafkaProducer
from kafka.partitioner import RoundRobinPartitioner

In [ ]:
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9095', 'localhost:9096'],
    api_version = (0, 10),
    partitioner = RoundRobinPartitioner(),
)

In [ ]:
producer.bootstrap_connected()

In [ ]:
from datetime import datetime
import json
from itertools import cycle

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        value_bytes = bytes(value, encoding = 'utf-8')
        x = producer_instance.send(topic_name, value = value_bytes)
        return True
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))
        return False

## purposely raise the error

Based on function `plus`, if input bigger than 10, it will throw error.

In [ ]:
# import confluent_kafka as ck

# source.consumer.committed([ck.TopicPartition('testing', 0)])[0].offset

In [ ]:
import time

while True:
    count = 10
    data = {'no': count, 'datetime': str(datetime.now())}
    #print(data)
    publish_message(
        producer, 'testing', 'streaming', json.dumps(data)
    )
    time.sleep(0.4)

In [ ]:
len(outputs)

Problem with `healing_batch`, if a message throw an exception, entire batch will not update.

In [ ]:
outputs